<a href="https://colab.research.google.com/github/Catatau52/Carteira_Quant/blob/main/Etapa_02_Premios_de_risco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from dateutil.relativedelta  import relativedelta
import os
from google.colab import drive
drive.mount('/content/drive')

In [ ]:


class premio_risco:

    def __init__(self, indicadores_dict, nome_premio, liquidez=0, caminho_dados = None, caminho_salvar_arquivo = '.'):

        self.indicadores = list(indicadores_dict.keys())
        self.ordem_indicadores = list(indicadores_dict.values())
        self.liquidez = liquidez
        self.nome_premio = nome_premio
        self.caminho_salvar_arquivo = caminho_salvar_arquivo

        if caminho_dados != None:

            os.chdir(caminho_dados)

    def pegando_dados_cotacoes(self):

        self.cotacoes = pd.read_parquet("cotacoes.parquet")
        self.cotacoes['id_dado'] = self.cotacoes['ticker'].astype(str) + "_" + self.cotacoes['data'].astype(str)
        self.cotacoes['data'] = pd.to_datetime(self.cotacoes['data']).dt.date

    def pegando_datas_possiveis(self):

        cot_petr = self.cotacoes[self.cotacoes['ticker'] == 'PETR4']

        cot_petr = cot_petr.sort_values('data', ascending = True)
        cot_petr = cot_petr.assign(year = pd.DatetimeIndex(cot_petr['data']).year)
        cot_petr = cot_petr.assign(month = pd.DatetimeIndex(cot_petr['data']).month)
        datas_final_mes = cot_petr.groupby(['year', 'month'])['data'].last()
        datas_final_mes = datas_final_mes.reset_index()

        self.datas_final_mes = datas_final_mes

    def filtrando_volume(self):

        dados_volume = pd.read_parquet("indicador_volume_mediano.parquet")
        dados_volume['id_dado'] = dados_volume['ticker'].astype(str) + "_" + dados_volume['data'].astype(str)
        dados_volume = dados_volume[['id_dado', 'valor']]
        dados_volume.columns = ['id_dado', 'volumeMediano']
        self.cotacoes = pd.merge(self.cotacoes, dados_volume, how = 'inner', on = 'id_dado')
        self.cotacoes = self.cotacoes[self.cotacoes['volumeMediano'] > self.liquidez]


    def pegando_indicadores(self):

        self.df_indicadores = []
        for indicador in self.indicadores:
            try:
                df = pd.read_parquet(f'{indicador}.parquet')
                df['data'] = pd.to_datetime(df['data']).dt.date
            except:
                df = None
            self.df_indicadores.append(df)

    def descobrindo_mes_inicial(self):
        datas = []
        for df in self.df_indicadores:
            indicador_sem_na = df.dropna()
            petr_indicador = indicador_sem_na.query('ticker == "PETR4"') #eu usei a petro como parametro pra saber a primeira data de qlqr indicador.
            data_minima = min(petr_indicador['data'])
            datas.append(data_minima)
        data_minima_geral = max(datas)  #maximo porque queremos a MAIOR data mínima entre os indicadores.
        self.data_minima_geral = data_minima_geral + relativedelta(months=+2)
         #vou colocar 2 meses depois da primeira empresa ter o indicador. isso pq os indicadores fundamentalistas no
        #1 mes da base podem ser muito escassos devido a maioria das empresas não terem soltado resultado ainda.
        self.lista_datas_final_mes = (self.datas_final_mes.query('data >= @self.data_minima_geral'))['data'].to_list()
        self.cotacoes_filtrado = self.cotacoes.query('data >= @self.data_minima_geral')

    def calculando_premios(self):

        colunas = ['primeiro_quartil', 'segundo_quartil', 'terceiro_quartil', 'quarto_quartil', 'universo']
        #colunas = ['primeiro_quartil', 'segundo_quartil', 'terceiro_quartil', 'quarto_quartil','quinto_quartil','universo']
        df_premios = pd.DataFrame(columns=colunas, index=self.lista_datas_final_mes)

        lista_dfs = [None] * 4 # inicializa lista para guardar dataframes de quartis
        #lista_dfs = [None] * 5

        for i, data in enumerate(self.lista_datas_final_mes):

            df_info_pontuais = self.cotacoes_filtrado[self.cotacoes_filtrado['data'] == data][['ticker', 'preco_fechamento_ajustado', 'volume_negociado']]

            #calculando rent da carteira anterior
            if i != 0:
                df_vendas = df_info_pontuais[['ticker', 'preco_fechamento_ajustado']]
                df_vendas.columns = ['ticker', 'preco_fechamento_ajustado_posterior']
                lista_retornos = []

                for zeta, df in enumerate(lista_dfs):
                    df_quartil = pd.merge(df, df_vendas, how='inner', on='ticker')
                    df_quartil['retorno'] = df_quartil['preco_fechamento_ajustado_posterior'] / df_quartil['preco_fechamento_ajustado'] - 1
                    retorno_quartil = df_quartil['retorno'].mean()
                    lista_retornos.append(retorno_quartil)
                    df_premios.loc[data, colunas[zeta]] = retorno_quartil

                df_premios.loc[data, 'universo'] = np.mean(np.array(lista_retornos))

            #pegando as novas carteiras
            df_info_pontuais['ranking_final'] = 0

            for alfa, indicador in enumerate(self.df_indicadores):

                indicador_na_data = indicador.loc[indicador['data'] == data, ['ticker', 'valor']].dropna()

                indicador_na_data.columns = ['ticker', f'indicador_{self.indicadores[alfa]}']
                df_info_pontuais = pd.merge(df_info_pontuais, indicador_na_data, how='inner', on='ticker')

            #filtrando empresas com varios tickers pro mais líquido
            df_info_pontuais['comeco_ticker'] = df_info_pontuais['ticker'].astype(str).str[0:4]
            df_info_pontuais.sort_values('volume_negociado', ascending=False, inplace=True)
            df_info_pontuais.drop_duplicates('comeco_ticker', inplace=True)
            df_info_pontuais.drop('comeco_ticker', axis=1, inplace=True)

            for alfa, ordem in enumerate(self.ordem_indicadores):
                crescente_condicao = ordem.lower() == 'crescente'
                df_info_pontuais[f'ranking_{alfa}'] = df_info_pontuais[f'indicador_{self.indicadores[alfa]}'].rank(ascending=crescente_condicao)
                df_info_pontuais['ranking_final'] += df_info_pontuais[f'ranking_{alfa}']

            df_info_pontuais.sort_values('ranking_final', ascending=True, inplace=True)

            empresas_por_quartil = len(df_info_pontuais) // 4
            sobra_empresas = len(df_info_pontuais) % 4
            #empresas_por_quartil = len(df_info_pontuais) // 5
            #sobra_empresas = len(df_info_pontuais) % 5

            # dividindo o dataframe em quartis
            lista_dfs[0] = df_info_pontuais.iloc[0: empresas_por_quartil]
            lista_dfs[1] = df_info_pontuais.iloc[empresas_por_quartil: (empresas_por_quartil * 2)]
            lista_dfs[2] = df_info_pontuais.iloc[(empresas_por_quartil * 2): (empresas_por_quartil * 3)]
            lista_dfs[3] = df_info_pontuais.iloc[(empresas_por_quartil * 3): ((empresas_por_quartil * 4) + sobra_empresas)]
            #lista_dfs[3] = df_info_pontuais.iloc[(empresas_por_quartil * 3): ((empresas_por_quartil * 4))]
            #lista_dfs[4] = df_info_pontuais.iloc[(empresas_por_quartil * 4): ((empresas_por_quartil * 5) + sobra_empresas )]

        df_premios['nome_premio'] = self.nome_premio
        df_premios['liquidez'] = self.liquidez
        df_premios.reset_index(names='data', inplace=True)
        #df_premios['id_premio'] = df_premios['nome_premio'].astype(str) + "_" + df_premios['liquidez'].astype(str) + "_" + df_premios['data'].astype(str)
        df_premios['id_premio'] = df_premios['nome_premio'].astype(str) + "_" + df_premios['data'].astype(str)
        df_premios.dropna(inplace=True)
        self.df_premios = df_premios

    def colocando_premio_na_base(self):

        #self.df_premios.to_parquet(f'{self.caminho_salvar_arquivo}/{self.nome_premio}_{self.liquidez}.parquet', index = False)
        self.df_premios.to_parquet(f'{self.caminho_salvar_arquivo}/{self.nome_premio}.parquet', index = False)


In [ ]:

if __name__ == "__main__":

    indicadores_dict = {
                          #'ROE': 'decrescente',
                          #'ROIC': 'decrescente',
                          #'EBIT_EV': 'decrescente',
                          #'EV_EBIT': 'decrescente',
                          #'L_P': 'decrescente',
                          #'P_L': 'decrescente',
                          #'LPA' : 'decrescente'
                          #'mm_7_40': 'decrescente',
                          #'ValorDeMercado_Cres': 'crescente',
                          #'ValorDeMercado_Decres': 'decrescente',
                          #'indicador_pl_db': 'crescente',
                          #'indicador_vpa': 'decrescente',
                          #'ebit_divliq': 'decrescente',
                          #'indicador_lucro_ev' : 'decrescente'
                          #'indicador_receita_ev' : 'decrescente'
                          #'indicador_lucro12m_ev' : 'decrescente'
                          #'indicador_receita12m_ev' : 'decrescente'
                          #'indicador_EV_Div' : 'decrescente'
                          #'My_EV_ebit' : 'decrescente'
                          #'My_ebit_EV' : 'decrescente'
                          #'indicador_ebit_Receita' : 'decrescente'
                          #'indicador_lucro_Receita' : 'decrescente'
                          #'momento_1_meses' : 'decrescente'
                          #'momento_3_meses' : 'decrescente'
                          #'momento_6_meses' : 'decrescente'
                          #'momento_12_meses' : 'decrescente'
                          #'p_vp' : 'crescente'
                          #'p_sr' : 'crescente'
                          #'indicador_p_ebit' : 'crescente'
                          #'indicador_Peg_ratio' : 'decrescente'
                          #'mm_9_21' : 'decrescente'
                          #'mm_13_26' : 'decrescente'
                          #'mm_13_48' : 'decrescente'
                          #'mm_14_40' : 'decrescente'
                          #'mm_20_100' : 'decrescente'
                          #'mm_50_200' : 'decrescente'
                          #'mm_20_80' : 'decrescente'
                          'mm_20_50' : 'decrescente'

                        }

    for nome in indicadores_dict.keys():
      premio = premio_risco(indicadores_dict,  liquidez = 1000000, nome_premio = nome,
                            caminho_dados=r'/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/base_dados_br',
                            caminho_salvar_arquivo=r'/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/base_dados_br/premios_risco_br'
                          ) #não pode ter \ no nome!!

      premio.pegando_dados_cotacoes()
      premio.pegando_datas_possiveis()
      premio.filtrando_volume()
      premio.pegando_indicadores()
      premio.descobrindo_mes_inicial()
      premio.calculando_premios()
      premio.colocando_premio_na_base()

      print(f'Data Frame {nome}.parquet gerado com sucesso!')

Data Frame mm_20_50.parquet gerado com sucesso!


In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/base_dados_br/premios_risco_br/indicador_Peg_ratio.parquet')
df.head(3)

,data,primeiro_quartil,segundo_quartil,terceiro_quartil,quarto_quartil,quinto_quartil,universo,nome_premio,liquidez,id_premio
0,2012-05-31,-0.093858,-0.038203,-0.075840,-0.099358,-0.045886,-0.070629,indicador_Peg_ratio,1000000,indicador_Peg_ratio_2012-05-31
1,2012-06-29,0.018762,0.031266,0.001214,-0.001041,0.024611,0.014963,indicador_Peg_ratio,1000000,indicador_Peg_ratio_2012-06-29
2,2012-07-31,0.052339,0.031488,0.073604,0.012301,0.014005,0.036748,indicador_Peg_ratio,1000000,indicador_Peg_ratio_2012-07-31


In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/base_dados_br/premios_risco_br/ROIC.parquet')
df.head(3)

,data,primeiro_quartil,segundo_quartil,terceiro_quartil,quarto_quartil,universo,nome_premio,liquidez,id_premio
0,2012-02-29,0.087861,0.104059,0.064780,0.079247,0.083987,ROIC,1000000,ROIC_2012-02-29
1,2012-03-30,0.051303,0.028402,0.011084,-0.008402,0.020597,ROIC,1000000,ROIC_2012-03-30
2,2012-04-30,0.015796,0.007394,-0.016010,-0.009657,-0.000619,ROIC,1000000,ROIC_2012-04-30


In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/base_dados_br/premios_risco_br/EBIT_EV.parquet')
df.head(3)

,data,primeiro_quartil,segundo_quartil,terceiro_quartil,quarto_quartil,universo,nome_premio,liquidez,id_premio
0,2012-02-29,0.072984,0.079416,0.093120,0.082507,0.082007,EBIT_EV,1000000,EBIT_EV_2012-02-29
1,2012-03-30,0.037242,0.017939,0.026551,-0.008513,0.018305,EBIT_EV,1000000,EBIT_EV_2012-03-30
2,2012-04-30,-0.016167,-0.019961,0.022091,-0.009518,-0.005889,EBIT_EV,1000000,EBIT_EV_2012-04-30
